# **STEP 5:** Preprocessing MRI DICOM images and convert to NifTi

In [1]:
#the python debugger is a library that provides an interactive source code debugger for Python programs
import pdb

import pydicom
import numpy as np
import cv2
import os
import shutil
import zipfile
from zipfile import ZipFile
import subprocess
import dcmrtstruct2nii
from dcmrtstruct2nii import dcmrtstruct2nii, list_rt_structs


Please cite:
Thomas Phil, Thomas Albrecht, Skylar Gay, & Mathis Ersted Rasmussen. (2023). Sikerdebaard/dcmrtstruct2nii: dcmrtstruct2nii v5 (Version v5). Zenodo. https://doi.org/10.5281/zenodo.4037864



## Normal Patients

### 5.1 Apply CLAHE and save as NIfTI

In [29]:
directory = '/notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/'

In [30]:
#shutil.rmtree('/notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_1-25/P1-25_RM_1_FOLLOW_UP/.ipynb_checkpoints')
#shutil.rmtree('/notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_1-25/P1-25_RM_1_TRATAMENTO/.ipynb_checkpoints')

In [31]:
# Iterate through the directory structure
for root, dirs, files in os.walk(directory):
    print("Visiting directory:", root)

    # Iterate over subdirectories
    for subdir in dirs:
        subdir_path = os.path.join(root, subdir)

        if subdir.startswith('.'):
            continue

        # Check if the subdirectory matches the naming criteria
        if "_RM_" in subdir or "RM_" in subdir or "MR_" in subdir:
            print(f"Processing directory: {subdir}")

            # Check if the subdirectory contains any subdirectory (nested directories)
            subdirs_inside = [d for d in os.listdir(subdir_path) if os.path.isdir(os.path.join(subdir_path, d))]

            if subdirs_inside and "ImageFus_" not in subdirs_inside:  # If there are nested directories
                print(f"Nested directories found in {subdir}. Proceeding with processing.")
                for subdir_name in subdirs_inside:
                    subdir_full_path = os.path.join(subdir_path, subdir_name)  # Full path to the nested directory

                    # Iterate over files in the nested directory
                    for file_name in os.listdir(subdir_full_path):
                        if file_name.lower().endswith('.dcm'):
                            file_path = os.path.join(subdir_full_path, file_name)
                            try:
                                # Read the DICOM file
                                ds = pydicom.dcmread(file_path, force=True)

                                # Normalize the pixel data
                                pixel_array = ds.pixel_array
                                min_val = np.min(pixel_array)
                                max_val = np.max(pixel_array)
                                normalized_pixel_array = np.uint8(((pixel_array - min_val) / (max_val - min_val)) * 255)

                                # Apply CLAHE
                                clahe = cv2.createCLAHE(clipLimit=5)
                                final_img = clahe.apply(normalized_pixel_array)

                                # Update the DICOM file with the processed pixel data
                                ds.PixelData = final_img.tobytes()
                                ds.BitsAllocated = 8
                                ds.BitsStored = 8
                                ds.HighBit = 7
                                ds.PixelRepresentation = 0

                                # Write the processed file to the output directory
                                #output_file_path = os.path.join(subdir_path, file_name)  # Save in the parent directory
                                pydicom.filewriter.write_file(ds.filename, ds)
                                print(f"CLAHE applied to {file_name}")
                                

                            except Exception as e:
                                print(f"Error processing DICOM file {file_path}: {e}")

            else:  # No nested directories
                print(f"No nested directories found in {subdir}. Proceeding with processing.")

                for file_name in os.listdir(subdir_path):
                    file_path = os.path.join(subdir_path, file_name)

                    # Check if it's a DICOM file (ends with .dcm)
                    if os.path.isfile(file_path) and file_name.lower().endswith('.dcm'):
                        try:
                            # Read the DICOM file
                            ds = pydicom.dcmread(file_path, force=True)

                            # Normalize the pixel data
                            pixel_array = ds.pixel_array
                            min_val = np.min(pixel_array)
                            max_val = np.max(pixel_array)
                            normalized_pixel_array = np.uint8(((pixel_array - min_val) / (max_val - min_val)) * 255)

                            # Apply CLAHE
                            clahe = cv2.createCLAHE(clipLimit=5)
                            final_img = clahe.apply(normalized_pixel_array)

                            # Update the DICOM file with the processed pixel data
                            ds.PixelData = final_img.tobytes()
                            ds.BitsAllocated = 8
                            ds.BitsStored = 8
                            ds.HighBit = 7
                            ds.PixelRepresentation = 0

                            # Write the processed file to the output directory
                            #output_file_path = os.path.join(subdir_path, file_name)  # Save in the parent directory
                            pydicom.filewriter.write_file(ds.filename, ds)
                            print(f"CLAHE applied to {file_name}")

                        except Exception as e:
                            print(f"Error processing DICOM file {file_path}: {e}")

        # Run the dcm2niix command
        output_dir = subdir_path  # Set the output directory for NIfTI files
        subprocess.run(['dcm2niix', '-z', 'y', '-o', output_dir, '-f', '%p_%s', subdir_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"NIfTI volume saved to: {output_dir}")



Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/
NIfTI volume saved to: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/REG_ImageFusions_7
NIfTI volume saved to: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_CT_1_TRATAMENTO
NIfTI volume saved to: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/REG_ImageFusions_9
Processing directory: P57_RM_1_TRATAMENTO
Nested directories found in P57_RM_1_TRATAMENTO. Proceeding with processing.
CLAHE applied to MR_164.dcm
CLAHE applied to MR_46.dcm
CLAHE applied to MR_334.dcm
CLAHE applied to MR_268.dcm
CLAHE applied to MR_232.dcm
CLAHE applied to MR_281.dcm
CLAHE applied to MR_101.dcm
CLAHE applied to MR_38.dcm
CLAHE applied to MR_10.dcm
CLAHE applied to MR_76.dcm
CLAHE applied to MR_88.dcm
CLAHE applied to MR_188.dcm
CLAHE applied to MR_119.dcm
CLAHE applied to MR_60.dcm
CLAHE applied to MR_204.dcm
CLAHE applied to MR_89.dcm
CLAHE applied to MR_217.dcm
CLAHE applied to MR_81.dcm
C

/tmp/ipykernel_16725/3135953173.py:36: RuntimeWarning: invalid value encountered in divide
  normalized_pixel_array = np.uint8(((pixel_array - min_val) / (max_val - min_val)) * 255)
/tmp/ipykernel_16725/3135953173.py:36: RuntimeWarning: invalid value encountered in cast
  normalized_pixel_array = np.uint8(((pixel_array - min_val) / (max_val - min_val)) * 255)


CLAHE applied to MR_271.dcm
CLAHE applied to MR_194.dcm
CLAHE applied to MR_270.dcm
CLAHE applied to MR_18.dcm
CLAHE applied to MR_90.dcm
CLAHE applied to MR_261.dcm
CLAHE applied to MR_15.dcm
CLAHE applied to MR_286.dcm
CLAHE applied to MR_117.dcm
CLAHE applied to MR_146.dcm
CLAHE applied to MR_298.dcm
CLAHE applied to MR_160.dcm
CLAHE applied to MR_315.dcm
CLAHE applied to MR_295.dcm
CLAHE applied to MR_48.dcm
CLAHE applied to MR_238.dcm
CLAHE applied to MR_191.dcm
CLAHE applied to MR_213.dcm
CLAHE applied to MR_53.dcm
CLAHE applied to MR_231.dcm
CLAHE applied to MR_237.dcm
CLAHE applied to MR_324.dcm
CLAHE applied to MR_159.dcm
CLAHE applied to MR_139.dcm
CLAHE applied to MR_23.dcm
CLAHE applied to MR_187.dcm
CLAHE applied to MR_122.dcm
CLAHE applied to MR_308.dcm
CLAHE applied to MR_221.dcm
CLAHE applied to MR_227.dcm
CLAHE applied to MR_243.dcm
CLAHE applied to MR_138.dcm
CLAHE applied to MR_62.dcm
CLAHE applied to MR_52.dcm
CLAHE applied to MR_251.dcm
CLAHE applied to MR_287.dcm


/tmp/ipykernel_16725/3135953173.py:36: RuntimeWarning: overflow encountered in scalar subtract
  normalized_pixel_array = np.uint8(((pixel_array - min_val) / (max_val - min_val)) * 255)


CLAHE applied to MR_185_3910.dcm
CLAHE applied to MR_143_4018.dcm
CLAHE applied to MR_191_4035.dcm
CLAHE applied to MR_271_4042.dcm
CLAHE applied to MR_14_4034.dcm
CLAHE applied to MR_97_3960.dcm
CLAHE applied to MR_128_4100.dcm
CLAHE applied to MR_5_4014.dcm
CLAHE applied to MR_235_3854.dcm
CLAHE applied to MR_106_3953.dcm
CLAHE applied to MR_127_4090.dcm
CLAHE applied to MR_160_3890.dcm
CLAHE applied to MR_47_3971.dcm
CLAHE applied to MR_99_3966.dcm
CLAHE applied to MR_111_3958.dcm
CLAHE applied to MR_250_3868.dcm
CLAHE applied to MR_20_3882.dcm
CLAHE applied to MR_115_4099.dcm
CLAHE applied to MR_172_4092.dcm
CLAHE applied to MR_149_3939.dcm
CLAHE applied to MR_146_4053.dcm
CLAHE applied to MR_194_4111.dcm
CLAHE applied to MR_7_4079.dcm
CLAHE applied to MR_213_4127.dcm
CLAHE applied to MR_131_4097.dcm
CLAHE applied to MR_272_4033.dcm
CLAHE applied to MR_148_3942.dcm
CLAHE applied to MR_26_3881.dcm
CLAHE applied to MR_260_4073.dcm
CLAHE applied to MR_92_3957.dcm
CLAHE applied to MR_1

In [32]:
for root, dirs, files in os.walk(directory):
    print("Visiting directory:", root)
    # Check if the current directory matches the naming criteria
    if "ImageFus_" in os.path.basename(root):
        print(f"Processing folder: {root}")
        parent_folder = os.path.dirname(root)  
        #print(parent_folder)
        for file in files:
            file_path = os.path.join(root, file)
            print(f"File path: {file_path}")
            
            # Move the file to the parent folder (P59_RM_1_FOLLOW_UP)
            destination_path = os.path.join(parent_folder, file)
            shutil.move(file_path, destination_path)
            print(f"Moved file to: {destination_path}")

        print(f"Done for folder '{root}'.")

Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/REG_ImageFusions_7
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_CT_1_TRATAMENTO
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_CT_1_TRATAMENTO/.ipynb_checkpoints
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/REG_ImageFusions_9
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/T1W_3D_FFECC_2
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/T1W_3D_FFE_3
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/.ipynb_checkpoints
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_Ni

In [33]:
for root, dirs, files in os.walk(directory):
    print("Visiting directory:", root)
    # Check if the current directory matches the naming criteria
    if "ImageFus_" in os.path.basename(root):
        print(f"Processing folder: {root}")
        if not os.listdir(root):  # If the directory is empty
            os.rmdir(root)  # Delete the folder
            print(f"Deleted empty folder: {root}")
        else:
            print(f"The folder '{root}' is not empty.")

        print(f"Done for folder '{root}'.")

Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/REG_ImageFusions_7
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_CT_1_TRATAMENTO
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_CT_1_TRATAMENTO/.ipynb_checkpoints
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/REG_ImageFusions_9
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/T1W_3D_FFECC_2
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/T1W_3D_FFE_3
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/.ipynb_checkpoints
Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_Ni

### 5.2 Delete DICOM files

In [34]:
for root, dirs, files in os.walk(directory, topdown=True):  # topdown=True to process directories first
    print("Visiting directory:", root)

    # Iterate over subdirectories
    for subdir in dirs:
        subdir_path = os.path.join(root, subdir)

        # If the subdirectory name matches criteria (e.g., '_RM_', 'RM_', 'MR_')
        if "_RM_" in subdir or "RM_" in subdir or "MR_" in subdir:
            print(f"Processing subdirectory: {subdir}")

            # Check if the subdirectory contains nested subdirectories
            subdirs_inside = [d for d in os.listdir(subdir_path) if os.path.isdir(os.path.join(subdir_path, d))]

            if subdirs_inside:  # If there are nested subdirectories
                print(f"Nested directories found in {subdir}. Proceeding with processing.")
                for subdir_name in subdirs_inside:
                    subdir_full_path = os.path.join(subdir_path, subdir_name)  # Full path to the nested directory
                    print(f"Deleting nested subdirectory: {subdir_full_path}")
                    shutil.rmtree(subdir_full_path)  # Delete the nested subdirectory
                    print(f"Deleted folder: {subdir_full_path}")

    # Now, process files in the current directory
    if "_RM_" in os.path.basename(root) or "RM_" in os.path.basename(root):
        print(f"Processing folder: {root}")
        
        for file in files:
            file_path = os.path.join(root, file)
            # Check if the file is not a NIfTI or JSON file and is not a directory
            if not (file.endswith(('.nii.gz', '.json', '.html')) or os.path.isdir(file_path)): 
                if not (file.startswith('REG_1_')):  # Skip REG_1_ files (like .dcm files you want to keep)
                    if file.endswith('.dcm'):  # Specifically target .dcm files for deletion
                        os.remove(file_path)
                        print(f"Deleted file: {file_path}")

        print(f"Done for folder '{root}'.")

Visiting directory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/
Processing subdirectory: P57_RM_1_TRATAMENTO
Nested directories found in P57_RM_1_TRATAMENTO. Proceeding with processing.
Deleting nested subdirectory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/T1W_3D_FFECC_2
Deleted folder: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/T1W_3D_FFECC_2
Deleting nested subdirectory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/T1W_3D_FFE_3
Deleted folder: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/T1W_3D_FFE_3
Deleting nested subdirectory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/.ipynb_checkpoints
Deleted folder: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57_RM_1_TRATAMENTO/.ipynb_checkpoints
Deleting nested subdirectory: /notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/P57

### 5.3 Zip

In [35]:
folderPath = "/notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/Patient_57/"
outputPath = "/notebooks/disk2/RT_Dataset_2/04_Zipped_Directories/01_NifTI_Dataset/Patient_57.zip"

In [36]:
# Zip Nifti directory
def zip_folder(folderPath, outputPath):
    with zipfile.ZipFile(outputPath, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folderPath):
            for file in files:
                zipf.write(os.path.join(root, file))
    print("--> Folder Zipped")
zip_folder(folderPath, outputPath)

--> Folder Zipped


Após o pré-processamento das imagens, fazer o co-registo no slicer e retirar, de seguida, as brain masks para utilizar depois no resto.

## Patients with missing CT

### 5.1. Convert Structures using 'dcmrtstruct2nii'

In [ ]:
# Convert RT Structures DICOM file to NifTi Volumes (one volume for each mask)
for folder in os.listdir(directory):
    if "_RM_" in folder and "TRATAMENTO" in folder:
        folder = os.path.join(directory, folder)
        for file in os.listdir(folder):
            if "RTSS.dcm" in file:
                struct_file = os.path.join(folder, file)
                mr_files = os.path.join(folder, 'MR_Images/')
                output_path = folder
                print(list_rt_structs(struct_file))
                dcmrtstruct2nii(struct_file, mr_files, output_path)
                print(f"Done Folder: {folder}")
            elif "DCMRT_Structures" in file:
                struct_file = os.path.join(folder, file)
                mr_files = os.path.join(folder, 'MR_Images/')
                output_path = folder
                print(list_rt_structs(struct_file))
                dcmrtstruct2nii(struct_file, mr_files, output_path)
                print(f"Done Folder: {folder}")

### 5.2 Apply CLAHE and save as NIfTI

In [ ]:
# Apply to each DICOM file in each subdirectory and save as NifTi
for root, dirs, files in os.walk(directory):
    for subdir in dirs:
        if "_RM_" in subdir and "TRATAMENTO" in subdir:
            subdir_path = os.path.join(directory, subdir, "MR_Images")

            # Apply CLAHE to each DICOM file in the subdirectory
            for file_name in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, file_name)
                ds = pydicom.dcmread(file_path)

                # Normalize the pixel data
                pixel_array = ds.pixel_array
                min_val = np.min(pixel_array)
                max_val = np.max(pixel_array)
                normalized_pixel_array = np.uint8(((pixel_array - min_val) / (max_val - min_val)) * 255)

                # Apply CLAHE
                clahe = cv2.createCLAHE(clipLimit=5)
                final_img = clahe.apply(normalized_pixel_array)

                ds.PixelData = final_img.tobytes()
                ds.BitsAllocated = 8  
                ds.BitsStored = 8     
                ds.HighBit = 7        
                ds.PixelRepresentation = 0  
                pydicom.filewriter.write_file(ds.filename, ds)
            print(f'CLAHE done for folder: {subdir}')

            # Call dcm2niix to convert modified DICOM to NIfTI
            output_dir = os.path.join(directory, subdir)

            # Run the dcm2niix command
            subprocess.run(['dcm2niix', '-z', 'y', '-o', output_dir, '-f', subdir, subdir_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

            shutil.rmtree(subdir_path)
            
            print(f'{subdir} NIfTI volume saved to: {output_dir}')
            
        elif "_RM_" in subdir and "TRATAMENTO" not in subdir:
                subdir_path = os.path.join(directory, subdir)
                
                # Apply CLAHE to each DICOM file in the subdirectory
                for file_name in os.listdir(subdir_path):
                    file_path = os.path.join(subdir_path, file_name)
                    ds = pydicom.dcmread(file_path)
    
                    # Normalize the pixel data
                    pixel_array = ds.pixel_array
                    min_val = np.min(pixel_array)
                    max_val = np.max(pixel_array)
                    normalized_pixel_array = np.uint8(((pixel_array - min_val) / (max_val - min_val)) * 255)
    
                    # Apply CLAHE
                    clahe = cv2.createCLAHE(clipLimit=5)
                    final_img = clahe.apply(normalized_pixel_array)
    
                    ds.PixelData = final_img.tobytes()
                    ds.BitsAllocated = 8  
                    ds.BitsStored = 8     
                    ds.HighBit = 7        
                    ds.PixelRepresentation = 0  
                    pydicom.filewriter.write_file(ds.filename, ds)
                print(f'CLAHE done for folder: {subdir}')
    
                # Call dcm2niix to convert modified DICOM to NIfTI
                output_dir = os.path.join(directory, subdir)
    
                # Run the dcm2niix command
                subprocess.run(['dcm2niix', '-z', 'y', '-o', output_dir, '-f', subdir, subdir_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

                    
                print(f'{subdir} NIfTI volume saved to: {output_dir}')

In [ ]:
# Delete MRI DICOM files from follow_up directories
for folder in  os.listdir(directory):
    if "_RM_" in folder and "TRATAMENTO" not in folder:
        folder_path = os.path.join(directory, folder)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            if not (file.endswith(('.nii.gz', '.json')) or os.path.isdir(file_path)):
                os.remove(file_path)
        print(f"Done for folder '{folder_path}'.")

In [ ]:
# Delete plan, dose and strutures DICOM files and image.nii.gz
for folder in os.listdir(directory):
    if "_RM_" in folder and "TRATAMENTO" in folder:
        output_path = os.path.join(directory, folder)
        print(output_path)
        for file in os.listdir(output_path):
            file_path = os.path.join(output_path, file)
            if os.path.isdir(file_path):
                shutil.rmtree(file_path)
                print(f"Deleted '{file_path}'.")
            elif file.endswith('image.nii.gz'):
                os.remove(file_path)
                print(f"Deleted '{file_path}'.")
            elif not (file.endswith(('.nii.gz', '.json'))) or os.path.isdir(file_path):
                os.remove(file_path)
                print(f"Deleted '{file_path}'.")

### 5.3 Zip

In [ ]:
folderPath = "/notebooks/disk2/RT_Dataset/05_Patients_NifTi/_Patient_25/"
outputPath = "/notebooks/disk2/RT_Dataset/01_Zipped_Directories/03_NifTi_Dataset/_Patient_25.zip"

In [ ]:
# Zip Nifti directory
def zip_folder(folderPath, outputPath):
    with zipfile.ZipFile(outputPath, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folderPath):
            for file in files:
                zipf.write(os.path.join(root, file))
    print("--> Folder Zipped")
zip_folder(folderPath, outputPath)